In [0]:
#Importing the libraries
import numpy  as np  
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import keras


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#Loading the data
x_train_images = pd.read_pickle('drive/My Drive/ML_miniproject_3/train_images.pkl')
x_test_images  = pd.read_pickle('drive/My Drive/ML_miniproject_3/test_images.pkl')
y_train_labels = pd.read_csv('drive/My Drive/ML_miniproject_3/train_labels.csv')

In [0]:
# Displaying the image
for id in range(1):   
    plt.title('Label: {}'.format(y_train_labels.iloc[id]['Category']))
    plt.imshow(x_train_images[id], cmap='gray')
    plt.show()

In [0]:
# Reshaping the data for processing
X_train = x_train_images.reshape(-1, 64, 64) # reshape 
X_test = x_test_images.reshape(-1, 64, 64) # reshape
Y_train = y_train_labels['Category']
Y_train = Y_train.values.reshape(-1, 1)

In [0]:
# This function gets the bounding boxes around each digit
def bounding_box(x):
    box = []
    for i in range(x.shape[0]):
        image = x[i].copy()

        # threshold and find contours
        # ret value set to threshold of 254
        ret, thresh = cv2.threshold(image,254,255,0)
        imagecon, contours, hierarchy = cv2.findContours(np.uint8(thresh), 0, 2)

        # finding minimum area rectangle covering contour
        min_area = []
        for c in contours:
            rect = cv2.minAreaRect(c)
            pos, size, orientation = rect
            area = max(size[0],size[1])**2
            # discarding boxes that cannot be possibly a digit
            if area > 49:
                min_area.append((area,[pos[0],pos[1],size[0],size[1],orientation]))
        
        # sorting the area, from largest
        min_area.sort(key=lambda x: x[0], reverse=True)
        min_area = list(list(zip(*min_area))[1])
        box.append(min_area)
    
    return box

In [0]:
# This function return coordinates of box around each digit
def coordinate(img, bounding_box, offset=0):
    _, image_threshold = cv2.threshold(img, 254, 255, 0)
    x_pos, y_pos, width, height, orientation = bounding_box
    box = cv2.boxPoints(((x_pos, y_pos), (width, height), orientation))
    box = np.int0(box)

    x_min = max(min(box[:,0]), 0)
    x_max = min(max(box[:,0]), image_threshold.shape[0])
    y_min = max(min(box[:,1]), 0)
    y_max = min(max(box[:,1]), image_threshold.shape[0])
    
    # bounding box without orientation
    digit = image_threshold[y_min:y_max, x_min:x_max].copy()
    
    # tightening up the bounding box 
    sum_x     = np.sum(digit,axis=0)
    nonzero_x = np.nonzero(sum_x)
    x_min    += np.amin(nonzero_x)
    x_max    -= (digit.shape[1] - np.amax(nonzero_x))

    sum_y     = np.sum(digit,axis=1)
    nonzero_y = np.nonzero(sum_y)
    y_min    += np.amin(nonzero_y)
    y_max    -= (digit.shape[0] - np.amax(nonzero_y))
    
    x_min = max(x_min-offset,0)
    x_max = min(x_max+offset,image_threshold.shape[0])
    y_min = max(y_min-offset,0)
    y_max = min(y_max+offset,image_threshold.shape[0])
    
    width  = x_max-x_min
    height = y_max-y_min
    
    return x_min, y_min, width, height

In [0]:
# Oriented bounding boxes for all digits

def preprocess(Image):
  
  box_train = bounding_box(Image)
  original_x = []
  proc_x = []
  proc_y = []
  box_train_proc =[]
  
  for i in range(Image.shape[0]):

      # there cannot be possibly a single digit, hence discard
      x, y, width, height, _ = box_train[i][0]
      if width >= 64 or height >= 64:
        print(i)
        continue

      # getting un-rotated digit and threshold
      x, y, w, h      = coordinate(Image[i], box_train[i][0])
      x, y, w, h      = int(x),int(y),int(w),int(h)
      _, image_thresh = cv2.threshold(Image[i, y:y+h, x:x+w], 254, 255, 0)  

      # padding for a square...
      max_width_height = max(w,h)
      if max_width_height > w:
        padding      = int((max_width_height-w)/2)
        image_thresh = np.pad(image_thresh, ((0,0),(padding,padding)), 'constant', constant_values=0)
      elif max_width_height > h:
        pad_amt      = int((max_width_height-h)/2)
        image_thresh = np.pad(image_thresh, ((padding,padding),(0,0)), 'constant', constant_values=0)

      # padding to have a border of two pixels of 28
      image_thresh = np.pad(image_thresh, 2, 'constant', constant_values=0)
      image_thresh = cv2.resize(image_thresh, (28,28))

      original_x.append(Image[i])
      proc_x.append(image_thresh)
      proc_y.append(Y_train[i])
      box_train_proc.append(box_train[i])

  original_x     = np.array(original_x)
  proc_x         = np.expand_dims(np.array(proc_x),axis=1)
  proc_y         = np.array(proc_y)
  box_train_proc = np.array(box_train_proc)

  
  return proc_x, proc_y

In [0]:
# Preprocessing of the training and testing images
X_train_processed, Y_train_processed = preprocess(X_train) 
X_test_processed, temp               = preprocess(X_test)
del(temp)

In [0]:
# Display the processed training and testing image
X_train_plot = X_train_processed.reshape(40000,28,28)
print(X_train_plot.shape)
print("Processed training images...")
for id in range(3):   
    plt.title('Label: {}'.format(y_train_labels.iloc[id]['Category']))
    plt.imshow(X_train_plot[id], cmap='gray')
    plt.show()

X_test_plot = X_test_processed.reshape(10000,28,28)
print(X_test_plot.shape)
print("Processed training images...")
for id in range(5):   
    plt.imshow(X_test_plot[id], cmap='gray')
    plt.show()



In [0]:
# Second processing of the images
preX_train, preX_test= [],[]

# Normalising the images
X_train_processed_N = X_train_processed/255
X_test_processed_N  = X_test_processed/255

#Training set
for image in X_train_processed_N:
    reshaped = image.reshape((784))
    preX_train.append(reshaped)
    
#Test set
for image in X_test_processed_N:
    reshaped = image.reshape((784))
    preX_test.append(reshaped)


preX_train = np.array(preX_train)
preX_test  = np.array(preX_test)

x_train = preX_train[:37500]
y_train = Y_train[:37500]
x_valid = preX_train[37500:]
y_valid = Y_train[37500:]
x_test  = preX_test

x_train = x_train.reshape(x_train.shape[0], 28, 28,1).astype('float32')
x_valid = x_valid.reshape(x_valid.shape[0], 28, 28,1).astype('float32')
x_test  = x_test.reshape(x_test.shape[0], 28, 28,1).astype('float32')

# Fitting the model
from keras.utils.np_utils import to_categorical
from  keras.utils import np_utils
y_binary = np_utils.to_categorical(y_train)
y_valid_binary= np_utils.to_categorical(y_valid)



In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler,ReduceLROnPlateau


datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        vertical_flip=False)  # randomly flip images
datagen.fit(x_train)

#Callbacks

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                                patience=5, 
                                                verbose=1, 
                                                factor=0.5, 
                                                min_lr=0.0001)
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

In [0]:
# Update the parameter values 
num_classes = 10
n_epochs = 15 
batch_size = 128

In [0]:
#vgg5 Model

from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D,Activation,BatchNormalization

def CNN_model():
 
  # creating the model
  model = Sequential()
  
  #1st 2 layer
  model.add(Conv2D(64, (3, 3), input_shape=(28, 28,1), activation=None, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  
  model.add(Conv2D(64, (3, 3), activation=None, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  
  model.add(MaxPooling2D((2, 2), strides=(2,2)))
  
  #2nd two layer
  model.add(Conv2D(128, (3, 3), activation=None, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  
  model.add(Conv2D(128, (3, 3), activation=None, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  
  model.add(MaxPooling2D((2, 2), strides=(2,2)))
  
  #1st 3 layer
  model.add(Conv2D(256, (3, 3), activation=None, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  
  model.add(Conv2D(256, (3,3), activation=None, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  
  model.add(Conv2D(256, (3, 3), activation=None, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  
  model.add(MaxPooling2D((2, 2), strides=(2,2)))
  
  #2st 3 layer
  model.add(Conv2D(512, (3, 3), activation=None, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  
  model.add(Conv2D(512, (3, 3), activation=None, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  
  model.add(Conv2D(512, (3,3), activation=None, padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  
  model.add(MaxPooling2D((2, 2), strides=(2,2)))
 
  
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dense(512, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))
  
  # Compiling the model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  return model

In [16]:
model1 = CNN_model()
filepath="best_weights1.hdf5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
history1 = model1.fit_generator(datagen.flow(x_train, y_binary, batch_size=batch_size),
                              steps_per_epoch=x_train.shape[0] // batch_size,
                              validation_data=(x_valid, y_valid_binary),
                              epochs = 2, callbacks=[learning_rate_reduction, checkpoint1])
                   

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
292/292 [==============================] - 45s 155ms/step - loss: 0.7462 - acc: 0.7779 - val_loss: 0.7082 - val_acc: 0.8168

Epoch 00001: val_acc improved from -inf to 0.81680, saving model to best_weights1.hdf5
Epoch 2/2
292/292 [==============================] - 39s 133ms/step - loss: 0.3535 - acc: 0.9094 - val_loss: 0.6946 - val_acc: 0.8372

Epoch 00002: val_acc improved from 0.81680 to 0.83720, saving model to best_weights1.hdf5


In [0]:
#source: https://github.com/CodeRiderViru/Hand-written-digit-recognition-/blob/master/mnist.ipynb

def CNN():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.20))
    
    model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes, activation='softmax'))
    
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [52]:
model2 = CNN()
filepath="best_weights2.hdf5"
checkpoint2 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
history2 = model2.fit_generator(datagen.flow(x_train,y_binary, batch_size=batch_size),steps_per_epoch=(len(x_train)//batch_size),
                               epochs = 30, validation_data = (x_valid,y_valid_binary),
                               callbacks=[learning_rate_reduction, checkpoint2])


Epoch 1/30
292/292 [==============================] - 20s 67ms/step - loss: 0.9380 - acc: 0.7051 - val_loss: 0.4148 - val_acc: 0.8984

Epoch 00001: val_acc improved from -inf to 0.89840, saving model to best_weights2.hdf5
Epoch 2/30
292/292 [==============================] - 17s 57ms/step - loss: 0.4596 - acc: 0.8809 - val_loss: 0.3693 - val_acc: 0.9216

Epoch 00002: val_acc improved from 0.89840 to 0.92160, saving model to best_weights2.hdf5
Epoch 3/30
292/292 [==============================] - 16s 56ms/step - loss: 0.3960 - acc: 0.9016 - val_loss: 0.3527 - val_acc: 0.9276

Epoch 00003: val_acc improved from 0.92160 to 0.92760, saving model to best_weights2.hdf5
Epoch 4/30
292/292 [==============================] - 16s 56ms/step - loss: 0.3620 - acc: 0.9106 - val_loss: 0.3552 - val_acc: 0.9260

Epoch 00004: val_acc did not improve from 0.92760
Epoch 5/30
292/292 [==============================] - 17s 57ms/step - loss: 0.3418 - acc: 0.9157 - val_loss: 0.3258 - val_acc: 0.9272

Epoch 00

In [19]:
model1.load_weights("best_weights1.hdf5")
Accuracy = model1.evaluate(x_valid, y_valid_binary, verbose=0)
print("Deep CNN Accuracy: %.2f%%" % (Accuracy[1]*100))

Deep CNN Accuracy: 83.72%


In [53]:

model2.load_weights("best_weights2.hdf5")
Accuracy = model2.evaluate(x_valid, y_valid_binary, verbose=0)
print("Deep CNN Accuracy: %.2f%%" % (Accuracy[1]*100))


Deep CNN Accuracy: 94.60%


In [54]:
results = np.zeros( (x_valid.shape[0],10) )
#results = results + model1.predict(x_valid) #x_val
results = results + model2.predict(x_valid) #x_val
#results = results + model3.predict(x_test) #x_val
#results = results + model4.predict(x_test) #x_val
#results = results + model5.predict(x_test) #x_val
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
y_valid_binary[0].shape


(10,)

In [0]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(results,y_valid)

In [0]:
results = np.zeros( (x_test.shape[0],10) )
#results = results + model1.predict(x_test) #x_val
results = results + model2.predict(x_test) #x_val
#results = results + model3.predict(x_test) #x_val
#results = results + model4.predict(x_test) #x_val
#results = results + model5.predict(x_test) #x_val
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
submission = pd.DataFrame({'ImageId': np.arange(1, len(results)+1), 'Label': results})
submission.to_csv('submission_0.05.csv', index=False)


In [0]:
Accuracy = model.evaluate(x_valid, y_valid_binary, verbose=0)
print("Deep CNN Accuracy: %.2f%%" % (Accuracy[1]*100))